In [2]:
import pandas as pd
import sqlite3
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns


conn = sqlite3.connect('database.db')
quiz = pd.read_sql("SELECT * FROM quiz", conn)
apprenant = pd.read_sql("SELECT * FROM apprenant", conn)
conn.close()

quiz[['date', 'time']] = quiz['datetime'].str.split(' ', expand=True)
quiz['date'] = pd.to_datetime(quiz['date'], dayfirst=False).dt.strftime('%y-%m-%d')
quiz['time'] = pd.to_datetime(quiz['time'], format='%H:%M:%S').dt.time
quiz.drop(['datetime'], axis=1, inplace=True)
quiz["type"] = quiz["type"].map({"facile":"F", "intermediaire":"I", "difficile":"D"})

apprenant["milieu"] = apprenant["milieu"].map({"rural":"R", "urbain":"U", "suburbain":"S"})

data = pd.merge(quiz, apprenant, how="inner", on="id_user")
data.drop(['id_x','id_y','nom','prenom'], axis=1, inplace=True)
data = data.set_index("id_user")
data = data.rename(columns={"methodeApprentissage":"MA","niveau_initial":"NV", "classe_sociale":"CS"})
data

,type,duree,score,date,time,age,CS,milieu,NV,sexe,MA
id_user,,,,,,,,,,,
2,F,113,14,23-05-19,11:35:23,20,c3,S,n4,M,m2
2,I,168,13,23-06-02,16:57:48,20,c3,S,n4,M,m2
2,D,265,11,23-07-15,20:26:38,20,c3,S,n4,M,m2
2,F,78,16,23-11-14,13:15:45,20,c3,S,n4,M,m2
2,I,126,15,23-12-03,17:22:35,20,c3,S,n4,M,m2
...,...,...,...,...,...,...,...,...,...,...,...
11,D,275,7,24-05-01,19:35:48,24,c1,R,n1,F,m1
11,D,255,9,24-05-07,20:45:58,24,c1,R,n1,F,m1
12,F,9,4,24-06-10,19:42:26,19,c3,U,n4,F,m1


In [ ]:
X = data[['age', 'CS', 'milieu', 'NV', 'sexe', 'type', 'duree']]
y = data['score']

X = pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': model.coef_})
coefficients

In [ ]:
coefficients_pivot = coefficients.set_index("Feature")
coefficients_pivot
plt.figure(figsize=(12, 8))
sns.heatmap(coefficients_pivot, annot=True, cmap='coolwarm', center=0)
plt.title('Heatmap des coefficients de la régression linéaire')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import sqlite3

conn = sqlite3.connect('database.db')
data = pd.read_sql("SELECT * FROM quiz", conn)

avg_scores = data.groupby('type')['score'].mean()

avg_scores.plot(kind='bar', figsize=(10, 6))
plt.title('Average Quiz Scores by Type')
plt.xlabel('Quiz Type')
plt.ylabel('Average Score')
plt.xticks(rotation=45)
plt.show()
data['datetime'] = pd.to_datetime(data['datetime'])

avg_scores_date = data.groupby(data['datetime'].dt.date)['score'].mean()

plt.plot(avg_scores_date.index, avg_scores_date.values, marker='o')
plt.title('Average Quiz Scores Over Time')
plt.xlabel('Date')
plt.ylabel('Average Score')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

plt.hist(data['score'], bins=10, color='skyblue', edgecolor='black')
plt.title('Distribution of Quiz Scores')
plt.xlabel('Score')
plt.ylabel('Frequency')
plt.show()

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

conn = sqlite3.connect('database.db')

query = "SELECT * FROM quiz q JOIN apprenant a ON q.id_user = a.id_user"
df = pd.read_sql_query(query, conn)

conn.close()

le = LabelEncoder()
df['type_encoded'] = le.fit_transform(df['type'])
df['classe_sociale_encoded'] = le.fit_transform(df['classe_sociale'])
df['milieu_encoded'] = le.fit_transform(df['milieu'])
df['niveau_initial_encoded'] = le.fit_transform(df['niveau_initial'])
df['sexe_encoded'] = le.fit_transform(df['sexe'])
df['methodeApprentissage_encoded'] = le.fit_transform(df['methodeApprentissage'])

plt.figure(figsize=(10, 6))
sns.countplot(x='type', data=df)
plt.title('Count of Quiz Types')
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(df['score'], bins=20, kde=True)
plt.title('Distribution of Quiz Scores')
plt.xlabel('Score')
plt.ylabel('Frequency')
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(df['duree'], bins=20, kde=True)
plt.title('Distribution of Quiz Duration')
plt.show()

plt.figure(figsize=(12, 8))
sns.lineplot(data=df, x='datetime', y='score', hue='type')
plt.title('Quiz Scores Over Time')
plt.xlabel('Datetime')
plt.ylabel('Score')
plt.show()

In [ ]:

avg_score_by_classe_sociale = df.groupby('classe_sociale')['score'].mean()
print("Average score by classe_sociale:")
print(avg_score_by_classe_sociale)

avg_score_by_milieu = df.groupby('milieu')['score'].mean()
print("Average score by milieu:")
print(avg_score_by_milieu)

avg_score_by_age = df.groupby('age')['score'].mean()
print("Average score by age:")
print(avg_score_by_age)

avg_score_by_niveau_initial = df.groupby('niveau_initial')['score'].mean()
print("Average score by niveau_initial:")
print(avg_score_by_niveau_initial)

avg_score_by_methodeApprentissage = df.groupby('methodeApprentissage')['score'].mean()
print("Average score by methodeApprentissage:")
print(avg_score_by_methodeApprentissage)

avg_score_by_sexe = df.groupby('sexe')['score'].mean()
print("Average score by sexe:")
print(avg_score_by_sexe)

type_dict = {'facile': 0, 'intermediaire': 1, 'difficile': 2}
df['type'] = df['type'].replace(type_dict)

plt.figure(figsize=(10, 6))
correlation = df.corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show


In [ ]:
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os

conn = sqlite3.connect('database.db')
cursor = conn.cursor()

cursor.execute("SELECT id FROM user WHERE username = ?", ("kamalc73",))
id_tuple = cursor.fetchone()


user_id = id_tuple[0]

cursor.execute("SELECT * FROM quiz WHERE id_user = ? ORDER BY datetime", (user_id,))
quiz_data = cursor.fetchall()

columns = [column[0] for column in cursor.description]
quiz_df = pd.DataFrame(quiz_data, columns=columns)

quiz_df['nb'] = range(1, len(quiz_df) + 1)

plt.figure(figsize=(10, 6))
sns.lineplot(x="nb", y="score", hue="type", markers=True, dashes=True, data=quiz_df)
plt.xlabel('Attempt Number')  
plt.ylabel('Score')           
plt.title('Quiz Scores Over Attempts') 
plt.grid(True)  
if not os.path.exists('static'):
    os.makedirs('static')
plot_path = os.path.join('static', 'plot.png')
plt.savefig(plot_path, bbox_inches='tight')
plt.show()
plt.close()


In [ ]:
import sqlite3

class statistique():
    def statistiquesAp(self,a="age"):
        conn = sqlite3.connect('database.db')
        apprenant = pd.read_sql("SELECT * FROM apprenant", conn)
        conn.close()
        plt.figure(figsize=(10, 6))
        sns.histplot(apprenant, x=a, discrete=True)

        plt.xlabel(a)
        plt.ylabel('Nombre d\'apprenants')
        plt.title(f'Distribution des apprenants par {a}')
        plt.show()
        plot_path = os.path.join('static', 'apprenant.png')
        plt.savefig(plot_path, bbox_inches='tight')
        plt.show()
        plt.close()

s = statistique()
s.statistiquesAp()


In [ ]:
conn = sqlite3.connect('database.db')
quiz = pd.read_sql("SELECT * FROM quiz", conn)
apprenant = pd.read_sql("SELECT * FROM apprenant", conn)
conn.close()
quiz.head()
apprenant.head()
data = pd.merge(quiz, apprenant, how="inner", on="id_user")
data.drop(['id_x','id_y','nom','prenom'], axis=1, inplace=True)
data = data.set_index("id_user")
data2 = data.groupby("age").describe()["score"]
mean_scores_by_ma = data.groupby('age')['score'].mean().reset_index()
mean_scores_by_ma
plt.figure(figsize=(12, 6))
sns.barplot(data=mean_scores_by_ma, x='age', y='score', palette='viridis')

plt.xlabel('age')
plt.ylabel('Moyenne des scores')
plt.title('Moyenne des scores des apprenants par age')

In [ ]:
data2.head()